<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Gmail - List most intersting emails
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Gmail/Gmail_Get_emails_stats_by_sender.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Gmail+-+Get+emails+stats+by+sender:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #gmail #productivity #naas_drivers #operations #automation #analytics #plotly

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

**Description:** This notebook analyses users' inbox and extracts a list of senders that the user is most interested in, depending on the user's opening rate of the sender's emails in the last two weeks.

## Input

### Import libraries

In [1]:
import naas
from naas_drivers import email
import pandas as pd
import numpy as np
import plotly.express as px

### Setup Variables
Create an application password following [this procedure](https://support.google.com/mail/answer/185833?hl=en)
- `username`: This variable stores the username or email address associated with the email account
- `password`: This variable stores the password or authentication token required to access the email account
- `smtp_server`: This variable represents the SMTP server address used for sending emails.
- `box`: This variable stores the name or identifier of the mailbox or folder within the email account that will be accessed.

In [2]:
username = "theuniverse.bg@gmail.com"
password = naas.secret.get("GMAIL_APP_PASSWORD")
smtp_server = "imap.gmail.com"
box = "INBOX"

## Model

### Connect to email box

In [3]:
emails = email.connect(username, password, username, smtp_server)

In [5]:
emails

### Get email list

In [4]:
df_emails = emails.get()

error: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'

### Filter emails for the last two weeks

In [ ]:
two_weeks_ago = pd.Timestamp.now() - pd.DateOffset(weeks=2)
df_emails['Date'] = pd.to_datetime(df_emails['Date'])
filtered_emails = df_emails[df_emails['Date'] >= two_weeks_ago]

## Fetch the message IDs for the filtered emails (message IDs contain information regarding the email's state (eg "Opened"))

In [ ]:
message_ids = filtered_emails['MessageId'].tolist()

## Check if each email in the list has been opened

In [ ]:
opened_senders = []
for message_id in message_ids:
    response = emails._mailbox.get_message_history(message_id)
    history = response['history']
    for event in history:
        if 'message' in event and 'labelsAdded' in event['message']:
            labels_added = event['message']['labelsAdded']
            if any(label['labelId'] == 'OPENED' for label in labels_added):
                sender = filtered_emails.loc[filtered_emails['MessageId'] == message_id, 'From'].iloc[0]
                opened_senders.append(sender)

## Get most viewed senders by counting the occurencies

In [ ]:
most_viewed_senders = pd.Series(opened_senders).value_counts().head(10)

## Output

### Print the list with the most viewed senders

In [ ]:
print("Most viewed senders (based on opened emails) for the last 2 weeks:")
print(most_viewed_senders)